<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 20</h1>
</div>

Today's Topics:

* NDArray:  Rust's answer to numpy
* A simple Neural Network using NDArray

# NDArray

* An $n$-dimensional (e.g. 1-D, 2-D, 3-D, ...) container for general elements and numerics.

* Similar to Python's Numpy but with important differences.

* Rich set of methods for creating views and slices into the array

* Provides a rich set of methods and mathematical operations

## Key similarities with Python Numpy (from [link](https://docs.rs/ndarray/latest/ndarray/doc/ndarray_for_numpy_users/index.html#similarities))

* Arrays have single element types
* Arrays can have arbitrary dimensions
* Arrays can have arbitrary strides
* Indexing starts at 0
* Default ordering is row-major (more on that below)
* Arithmetic operators (+, -, \*, /) perform element-wise operations
* Arrays that are not views are contiguous in memory
* Many cheap operations that return views into the array instead of copying data




## Some important differences from Numpy

* Numpy arrays and views are all mutable and can all change the contents of an array.  
* NDarrays have:
    * flavors that can change contents, 
    * flavors that cannot, and 
    * flavors that make copies when things change.
* Numpy arrays are always dynamic in their number of dimensions.  NDarrays can be static or dynamic.
* Slices with negative steps behave differently (more on that later)



## To use it

To add latest `ndarray` crate.

```sh
% cargo add ndarray
```

or manually add

```
[dependencies]  
ndarray="0.15.6"  
```

or later in your Cargo.toml file


## Why use NDarray over Vec or array?

It is easier to do a bunch of things like:

* Data Cleaning and Preprocessing offering functions like slicing and fill
* Statistics built in (lots of math functions come with it)
* Machine learning: Used in many of the ML libraries written in Rust
* Linear Algebra: Built in methods like matrix inversion, multiplication and decomposition.

Let's look at some example usage.

In [2]:
// This is required in a Jupyter notebook.
// For a cargo project, you would add it to your Cargo.toml file.
:dep ndarray = { version = "^0.15.6" }

use ndarray::prelude::*;

fn main() {
    let a = array![         // handy macro for creating arrays
                [1.,2.,3.], 
                [4.,5.,6.],
            ]; 
    assert_eq!(a.ndim(), 2);         // get the number of dimensions of array a
    assert_eq!(a.len(), 6);          // get the number of elements in array a
    assert_eq!(a.shape(), [2, 3]);   // get the shape of array a
    assert_eq!(a.is_empty(), false); // check if the array has zero elements

    println!("Print the array with debug formatting:");
    println!("{:?}", a);

    println!("\nPrint the array with display formatting:");
    println!("{}", a);
}

main();

Print the array with debug formatting:
[[1.0, 2.0, 3.0],
 [4.0, 5.0, 6.0]], shape=[2, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2

Print the array with display formatting:
[[1, 2, 3],
 [4, 5, 6]]


For a side by side comparison NumPy, see https://docs.rs/ndarray/latest/ndarray/doc/ndarray_for_numpy_users/index.html



### Array Creation

|NumPy | ndarray | Notes |
|:-:|:-:|:-:|
| np.array([[1.,2.,3.], [4.,5.,6.]])| array![[1.,2.,3.], [4.,5.,6.]] or arr2(&[[1.,2.,3.], [4.,5.,6.]]) | 2×3 floating-point array literal|
|np.arange(0., 10., 0.5) | Array::range(0., 10., 0.5) | create a 1-D array with values 0., 0.5, …, 9.5|
|np.linspace(0., 10., 11) |	Array::linspace(0., 10., 11) | create a 1-D array with 11 elements with values 0., …, 10.|
|np.logspace(2.0, 3.0, num=4, base=10.0) | Array::logspace(10.0, 2.0, 3.0, 4) | create a 1-D array with 4 logarithmically spaced elements with values 100., 215.4, 464.1, 1000.|
|np.geomspace(1., 1000., num=4) | Array::geomspace(1e0, 1e3, 4) | create a 1-D array with 4 geometrically spaced elements from 1 to 1,000 inclusive: 1., 10., 100., 1000.|
|np.ones((3, 4, 5)) | Array::ones((3, 4, 5)) | create a 3×4×5 array filled with ones (inferring the element type)|
|np.zeros((3, 4, 5)) | Array::zeros((3, 4, 5)) | create a 3×4×5 array filled with zeros (inferring the element type)|
|np.zeros((3, 4, 5), order='F')	| Array::zeros((3, 4, 5).f()) | create a 3×4×5 array with Fortran (column-major) memory layout filled with zeros (inferring the element type)|
|np.zeros_like(a, order='C') | Array::zeros(a.raw_dim()) | create an array of zeros of the shape shape as a, with row-major memory layout (unlike NumPy, this infers the element type from context instead of duplicating a’s element type)|
|np.full((3, 4), 7.) | Array::from_elem((3, 4), 7.) | create a 3×4 array filled with the value 7.|
|np.array([1, 2, 3, 4]).reshape((2, 2)) | Array::from_shape_vec((2, 2), vec![1, 2, 3, 4])? or .into_shape((2,2).unwrap()| create a 2×2 array from the elements in the Vec|

In [4]:
let a:Array<f64, Ix1> = Array::linspace(0., 4.5, 10);
println!("{:?}", a);

let b:Array<f64, _> = a.into_shape((2,5)).unwrap();
println!("\n{:?}", b);

let c:Array<f64, _> = Array::zeros((3,4).f());
println!("\n{:?}", c);

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5], shape=[10], strides=[1], layout=CFcf (0xf), const ndim=1

[[0.0, 0.5, 1.0, 1.5, 2.0],
 [2.5, 3.0, 3.5, 4.0, 4.5]], shape=[2, 5], strides=[5, 1], layout=Cc (0x5), const ndim=2

[[0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0]], shape=[3, 4], strides=[1, 3], layout=Ff (0xa), const ndim=2


Good overview at this [TDS Post](https://towardsdatascience.com/the-ultimate-ndarray-handbook-mastering-the-art-of-scientific-computing-with-rust-ef5ab767212a/#672d), but subscription required.

In [2]:
:dep ndarray = { version = "^0.15.6" }

// Not working on Jupyter notebook
//:dep ndarray-rand = { version = "^0.15.0" }

use ndarray::{Array, ShapeBuilder};

// Not working on Jupyter notebook
//use ndarray_rand::RandomExt;
//use ndarray_rand::rand_distr::Uniform;


// Ones

let ones = Array::<f64, _>::ones((1, 4));
println!("{:?}", ones); 

// Output:
// [[1.0, 1.0, 1.0, 1.0]], shape=[1, 4], strides=[4, 1], layout=CFcf (0xf), const ndim=2

// Range

let range = Array::<f64, _>::range(0., 5., 1.);
println!("{:?}", range); 

// Output:
// [0.0, 1.0, 2.0, 3.0, 4.0], shape=[5], strides=[1], layout=CFcf (0xf), const ndim=1

// Linspace

let linspace = Array::<f64, _>::linspace(0., 5., 5);
println!("{:?}", linspace); 

// Output:
// [0.0, 1.25, 2.5, 3.75, 5.0], shape=[5], strides=[1], layout=CFcf (0xf), const ndim=1

// Fill

let mut ones = Array::<f64, _>::ones((1, 4));
ones.fill(0.);
println!("{:?}", ones); 

// Output:
// [[0.0, 0.0, 0.0, 0.0]], shape=[1, 4], strides=[4, 1], layout=CFcf (0xf), const ndim=2

// Eye -- Identity Matrix

let eye = Array::<f64, _>::eye(4);
println!("{:?}", eye); 

// Output:
// [[1.0, 0.0, 0.0, 0.0],
// [0.0, 1.0, 0.0, 0.0],
// [0.0, 0.0, 1.0, 0.0],
// [0.0, 0.0, 0.0, 1.0]], shape=[4, 4], strides=[4, 1], layout=Cc (0x5), const ndim=2

// Random

// Not working on Jupyter notebook

//let random = Array::random((2, 5), Uniform::new(0., 10.));
//println!("{:?}", random);

// Output:
// [[9.375493735188611, 4.088737328406999, 9.778579742815943, 0.5225866490310649, 1.518053969762827],
//  [9.860829919571666, 2.9473768443117, 7.768332993584486, 7.163926861520167, 9.814750664983297]], shape=[2, 5], strides=[5, 1], layout=Cc (0x5), const ndim=2

[[0.0, 0.0, 0.0, 0.0]], shape=[1, 4], strides=[1, 1], layout=CFcf (0xf), const ndim=2
[[1.0, 1.0, 1.0, 1.0]], shape=[1, 4], strides=[4, 1], layout=CFcf (0xf), const ndim=2
[0.0, 1.0, 2.0, 3.0, 4.0], shape=[5], strides=[1], layout=CFcf (0xf), const ndim=1
[0.0, 1.25, 2.5, 3.75, 5.0], shape=[5], strides=[1], layout=CFcf (0xf), const ndim=1
[[0.0, 0.0, 0.0, 0.0]], shape=[1, 4], strides=[4, 1], layout=CFcf (0xf), const ndim=2
[[1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0]], shape=[4, 4], strides=[4, 1], layout=Cc (0x5), const ndim=2


## What does the printout mean?

* The values of the array
* The shape of the array, most important dimension first
* The stride (always 1 for arrays in the last dimension, but can be different for views)
* The layout (storage order, view order)
* The number of dimensions

## Indexing and Slicing

|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|a[-1] | a[a.len() - 1] | access the last element in 1-D array a|
|a[1, 4] | a[[1, 4]] | access the element in row 1, column 4|
|a[1] or a[1, :, :] | a.slice(s![1, .., ..]) or a.index_axis(Axis(0), 1) | get a 2-D subview of a 3-D array at index 1 of axis 0|
|a[0:5] or a[:5] or a[0:5, :] | a.slice(s![0..5, ..]) or a.slice(s![..5, ..]) or a.slice_axis(Axis(0), Slice::from(0..5)) | get the first 5 rows of a 2-D array|
|a[-5:] or a[-5:, :] | a.slice(s![-5.., ..]) or a.slice_axis(Axis(0), Slice::from(-5..)) | get the last 5 rows of a 2-D array|
|a[:3, 4:9]	| a.slice(s![..3, 4..9]) | columns 4, 5, 6, 7, and 8 of the first 3 rows|
|a[1:4:2, ::-1]	| a.slice(s![1..4;2, ..;-1]) | rows 1 and 3 with the columns in reverse order|

## The `s![]` slice macro

* The `s![]` macro in Rust's ndarray crate is a convenient way to create slice specifications for array operations. 

* It's used to create a `SliceInfo` object that describes how to slice or view an array.

Here's how it works:

1. The `s![]` macro is used to create slice specifications that are similar to Python's slice notation
2. It's commonly used with methods like `slice()`, `slice_mut()`, and other array view operations
3. Inside the macro, you can specify ranges and steps for each dimension

For example:

```rust
let mut slice = array.slice_mut(s![1.., 0, ..]);
```

This creates a slice that:
- Takes all elements from index 1 onwards in the first dimension (`1..`)
- Takes only index 0 in the second dimension (`0`)
- Takes all elements in the third dimension (`..`)

The syntax inside `s![]` supports several patterns:
- `..` - take all elements in that dimension
- `start..end` - take elements from start (inclusive) to end (exclusive)
- `start..=end` - take elements from start (inclusive) to end (inclusive)
- `start..` - take elements from start (inclusive) to the end
- `..end` - take elements from the beginning to end (exclusive)
- `index` - take only that specific index

For example:
```rust
// Take every other element in the first dimension
s![..;2]

// Take elements 1 to 3 in the first dimension, all elements in the second
s![1..4, ..]

// Take elements from index 2 to the end, with step size 2
s![2..;2]

// Take specific indices
s![1, 2, 3]
```



<div align="center">
    <img src="figs/tds_array_slicing.webp" width="40%">
</div>

From [TDS Post](https://towardsdatascience.com/the-ultimate-ndarray-handbook-mastering-the-art-of-scientific-computing-with-rust-ef5ab767212a/#672d).

In [5]:
use ndarray::{s};
{
    // Create a 3-dimensional array (2x3x4)
    let mut array = Array::from_shape_fn((2, 3, 4), |(i, j, k)| {
        (i * 100 + j * 10 + k) as f32
    });

    // Print the original 3-dimensional array
    println!("Original 3D array:\n{:?}", array);

    // Create a 2-dimensional slice (taking the first 2D layer)
    let mut slice = array.slice_mut(s![1.., 0, ..]);

    // Print the 2-dimensional slice
    println!("2D slice:\n{:?}", slice);

    // Create a 1-dimensional slice (taking the first 2D and 3D layer)
    let slice2 = array.slice(s![1, 0, ..]);

    // Print the 1-dimensional slice
    println!("1D slice:\n{:?}", slice2);

}

Original 3D array:
[[[0.0, 1.0, 2.0, 3.0],
  [10.0, 11.0, 12.0, 13.0],
  [20.0, 21.0, 22.0, 23.0]],

 [[100.0, 101.0, 102.0, 103.0],
  [110.0, 111.0, 112.0, 113.0],
  [120.0, 121.0, 122.0, 123.0]]], shape=[2, 3, 4], strides=[12, 4, 1], layout=Cc (0x5), const ndim=3
2D slice:
[[100.0, 101.0, 102.0, 103.0]], shape=[1, 4], strides=[0, 1], layout=CFcf (0xf), const ndim=2
1D slice:
[100.0, 101.0, 102.0, 103.0], shape=[4], strides=[1], layout=CFcf (0xf), const ndim=1


()

2D and 3D datatypes, again from [TDS Post](https://towardsdatascience.com/the-ultimate-ndarray-handbook-mastering-the-art-of-scientific-computing-with-rust-ef5ab767212a/#672d).

In [3]:
use ndarray::{array, Array, Array2, Array3, ShapeBuilder};

// 1D array
let array_d1 = Array::from_vec(vec![1., 2., 3., 4.]);
println!("{:?}", array_d1);

// Output:
// [1.0, 2.0, 3.0, 4.0], shape=[4], strides=[1], layout=CFcf (0xf), const ndim=1

// or 

let array_d11 = Array::from_shape_vec((1, 4), vec![1., 2., 3., 4.]);
println!("{:?}", array_d11.unwrap());

// Output:
// [[1.0, 2.0, 3.0, 4.0]], shape=[1, 4], strides=[4, 1], layout=CFcf (0xf), const ndim=2

// 2D array

let array_d2 = array![
    [-1.01,  0.86, -4.60,  3.31, -4.81],
    [ 3.98,  0.53, -7.04,  5.29,  3.55],
    [ 3.30,  8.26, -3.89,  8.20, -1.51],
    [ 4.43,  4.96, -7.66, -7.33,  6.18],
    [ 7.31, -6.43, -6.16,  2.47,  5.58],
];

// or

let array_d2 = Array::from_shape_vec((2, 2), vec![1., 2., 3., 4.]);
println!("{:?}", array_d2.unwrap());

// Output:
// [[1.0, 2.0],
// [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2

// or

let mut data = vec![1., 2., 3., 4.];
let array_d21 = Array2::from_shape_vec((2, 2), data);

// 3D array

let mut data = vec![1., 2., 3., 4.];
let array_d3 = Array3::from_shape_vec((2, 2, 1), data);
println!("{:?}", array_d3);

// Output:
// [[[1.0],
//  [2.0]],
//  [[3.0],
//  [4.0]]], shape=[2, 2, 1], strides=[2, 1, 1], layout=Cc (0x5), const ndim=3

[1.0, 2.0, 3.0, 4.0], shape=[4], strides=[1], layout=CFcf (0xf), const ndim=1
[[1.0, 2.0, 3.0, 4.0]], shape=[1, 4], strides=[4, 1], layout=CFcf (0xf), const ndim=2
[[1.0, 2.0],
 [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2
Ok([[[1.0],
  [2.0]],

 [[3.0],
  [4.0]]], shape=[2, 2, 1], strides=[2, 1, 1], layout=Cc (0x5), const ndim=3)


## Reshaping

<div align="center">
    <img src="figs/tds_reshape.webp" width="50%">
</div>

From [TDS Post](https://towardsdatascience.com/the-ultimate-ndarray-handbook-mastering-the-art-of-scientific-computing-with-rust-ef5ab767212a/#672d).

<div align="center">
    <img src="figs/tds_flatten.webp" width="50%">
</div>

From [TDS Post](https://towardsdatascience.com/the-ultimate-ndarray-handbook-mastering-the-art-of-scientific-computing-with-rust-ef5ab767212a/#672d).

## Examining the array parameters

|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|np.ndim(a) or a.ndim | a.ndim() | get the number of dimensions of array a|
|np.size(a) or a.size | a.len()	| get the number of elements in array a|
|np.shape(a) or a.shape	| a.shape() or a.dim() | get the shape of array a|
|a.shape[axis] | a.len_of(Axis(axis)) | get the length of an axis|
|a.strides | a.strides() | get the strides of array a|
|np.size(a) == 0 or a.size == 0	| a.is_empty() | check if the array has zero elements|


## Simple Math

|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|a.transpose() or a.T | a.t() or a.reversed_axes() | transpose of array a (view for .t() or by-move for .reversed_axes())|
|mat1.dot(mat2) | mat1.dot(&mat2) |2-D matrix multiply|
|mat.dot(vec) | mat.dot(&vec) | 2-D matrix dot 1-D column vector|
|vec.dot(mat) | vec.dot(&mat) | 1-D row vector dot 2-D matrix |
|vec1.dot(vec2) | vec1.dot(&vec2) | vector dot product|
|a * b, a + b, etc. | a * b, a + b, etc. | element-wise arithmetic operations|
|a\**3 | a.mapv(\|a\| a.powi(3)) | element-wise power of 3|
|np.sqrt(a) | a.mapv(f64::sqrt) | element-wise square root for f64 array |
|(a>0.5) | a.mapv(\|a\| a > 0.5)| array of bools of same shape as a with true where a > 0.5 and false elsewhere|
|np.sum(a) or a.sum() | a.sum() | sum the elements in a|
| np.sum(a, axis=2) or a.sum(axis=2) | a.sum_axis(Axis(2)) | sum the elements in a along axis 2|
|np.mean(a) or a.mean() | a.mean().unwrap() | calculate the mean of the elements in f64 array a|
|np.mean(a, axis=2) or a.mean(axis=2) | a.mean_axis(Axis(2)) | calculate the mean of the elements in a along axis 2|
|np.allclose(a, b, atol=1e-8) | a.abs_diff_eq(&b, 1e-8) |  check if the arrays’ elementwise differences are within an absolute tolerance (it requires the approx feature-flag)|
| np.diag(a) | a.diag() | view the diagonal of a|


### mapv vs map:  mapv iterates over the values of the array, map iterates over mutable references of the array

### Array0 to Array6 also defined in addition to Array as special cases with fixed dimensions instead dynamically defined dimensions.

In [6]:
{
    // Create a 2-dimensional array (3x4)
    let mut a = Array::from_shape_fn((3, 4), |(j, k)| {
        (j * 10 + k) as f32
    });

    // Print the original 2-dimensional array
    println!("Original 2D array:\n{:?}", a);

    let b = a * 2.0;
    println!("\nb:\n{:?}", b);

    let c = b.mapv(|v| v>24.8);
    println!("\nc:\n{:?}", c);
}

Original 2D array:
[[0.0, 1.0, 2.0, 3.0],
 [10.0, 11.0, 12.0, 13.0],
 [20.0, 21.0, 22.0, 23.0]], shape=[3, 4], strides=[4, 1], layout=Cc (0x5), const ndim=2

b:
[[0.0, 2.0, 4.0, 6.0],
 [20.0, 22.0, 24.0, 26.0],
 [40.0, 42.0, 44.0, 46.0]], shape=[3, 4], strides=[4, 1], layout=Cc (0x5), const ndim=2

c:
[[false, false, false, false],
 [false, false, false, true],
 [true, true, true, true]], shape=[3, 4], strides=[4, 1], layout=Cc (0x5), const ndim=2


()

## Type Conversions

* std::convert::From ensures lossless, safe conversions at compile-time and is generally recommended.
* std::convert::TryFrom can be used for potentially unsafe conversions. It will return a Result which can be handled or unwrap()ed to panic if any value at runtime cannot be converted losslessly.


|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|a.astype(np.float32) | a.mapv(\|x\| f32::from(x)) | convert array to f32.  Only use if can't fail |
|a.astype(np.int32) | a.mapv(\|x\| i32::from(x)) | convert array to i32.  Only use if can't fail |
|a.astype(np.uint8) | a.mapv(\|x\| u8::try_from(x).unwrap()) | try to convert to u8 array, panic if any value cannot be converted lossless at runtime (e.g. negative value)|
|a.astype(np.int32) | a.mapv(\|x\| x as i32) | convert to i32 array with “saturating” conversion; care needed because it can be a lossy conversion or result in non-finite values!|



## Basic Linear Algebra

Provided by `NDArray`.

### Transpose

<div align="center">
    <img src="figs/tds_transpose.webp" width="50%">
</div>

In [ ]:
let array_d2 = Array::from_shape_vec((2, 2), vec![1., 2., 3., 4.]);
println!("{:?}", array_d2.unwrap());

// Output
// [[1.0, 2.0],
//  [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2)

let binding = array_d2.expect("Expect 2d matrix");

let array_d2t = binding.t();
println!("{:?}", array_d2t);

// Output
// [[1.0, 3.0],
//  [2.0, 4.0]], shape=[2, 2], strides=[1, 2], layout=Ff (0xa), const ndim=2


### Matrix Multiplication

In [5]:
use ndarray::{array, Array2};

let a: Array2<f64> = array![[3., 2.], [2., -2.]];
let b: Array2<f64> = array![[3., 2.], [2., -2.]];
let c = a.dot(&b);
print!("{:?}", c);

// Output
// [[13.0, 2.0],
//  [2.0, 8.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2


 [2.0, 8.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2[[13.0, 2.0],


## Linear Algebra with `ndarray-linalg`

* The crate [`ndarray-linalg`](https://crates.io/crates/ndarray-linalg) implements more advanced lineary algebra operations that comes with `NDArray`.

* It relies on a native linear algebra library like `OpenBLAS` and can be tricky to configure.

* We show it here just for reference.

|NumPy | ndarray | Notes|
|:-:|:-:|:-:|
|numpy.linalg.inv(a) | a.inv() | Invert matrix a.  Must be square|
|numpy.linalg.eig(a) | a.eig() | Compute eigenvalues and eigenvectors of matrix.  Must be square|
|numpy.linalg.svd(a) | a.svd(true, true) | Compute the Singular Value Decomposition of matrix|
|numpy.linalg.det(a) | a.det() | Compute the determinant of a matrix. Must be square|

In [7]:
//```rust
:dep ndarray = { version = "^0.15" }
// This is the MAC version
// See ./README.md for setup instructions
:dep ndarray-linalg = { version = "^0.16", features = ["openblas-system"] }
// This is the linux verison
//:dep ndarray-linalg = { version = "^0.15", features = ["openblas"] }

use ndarray::array;
use ndarray_linalg::*;

{
    // Create a 2D square matrix (3x3)
    let matrix = array![
        [1.0, 2.0, 3.0],
        [0.0, 1.0, 4.0],
        [5.0, 6.0, 0.0]
    ];

    // Compute the eigenvalues and eigenvectors
    match matrix.eig() {
        Ok((eigenvalues, eigenvectors)) => {
            println!("Eigenvalues:\n{:?}", eigenvalues);
            println!("Eigenvectors:\n{:?}", eigenvectors);
        },
        Err(e) => println!("Failed to compute eigenvalues and eigenvectors: {:?}", e),
    }
}
//```

The type of the variable c was redefined, so was lost.
The type of the variable b was redefined, so was lost.


Eigenvalues:
[Complex { re: 7.256022422687388, im: 0.0 }, Complex { re: -0.026352822204034426, im: 0.0 }, Complex { re: -5.229669600483354, im: 0.0 }], shape=[3], strides=[1], layout=CFcf (0xf), const ndim=1
Eigenvectors:
[[Complex { re: -0.4992701697014973, im: 0.0 }, Complex { re: -0.7576983872742932, im: 0.0 }, Complex { re: -0.22578016277159085, im: 0.0 }],
 [Complex { re: -0.4667420094775666, im: 0.0 }, Complex { re: 0.6321277120502754, im: 0.0 }, Complex { re: -0.526348454767688, im: 0.0 }],
 [Complex { re: -0.7299871192254567, im: 0.0 }, Complex { re: -0.162196515314045, im: 0.0 }, Complex { re: 0.8197442419819131, im: 0.0 }]], shape=[3, 3], strides=[1, 3], layout=Ff (0xa), const ndim=2


()

The same code is provided as a [cargo project](./neural_network/).

## Python Numpy Reference

For reference you can look at [mnist_fcn.ipynb](./mnist_fcn.ipynb) which implements and 
trains the network with only numpy matrices, but does use PyTorch dataset loaders for conciseness.


# NDArray and Neural networks

* Simple neural networks can be easily represented by matrix arithmentic.

* Let's see how we can build a relatively simple neural network to recognize handwritten digits.


## Biological Neuron

An _artificial neuron_is loosely modeled on a biological neuron.

<div align="center">
    <img src="./figs/neuron.png" width="75%">
</div>


From Stanford's [cs231n](https://cs231n.github.io/neural-networks-1/)

* The dendrites carry impulses from other neurons of different distances.
* Once the collective firing rate of the impulses exceed a certain threshold,
  the neuron fires its own pulse through the axon to other neurons.

## Artificial Neuron

The artificial neuron is loosely based on the biological one.

<div>
    <img src="figs/neuron_model.jpeg" width="75%">
</div>

From Stanford's [cs231n](https://cs231n.github.io/neural-networks-1/).

The artifical neuron:

* collects one or more inputs, 
* each multiplied by a unique weight,
* sums the weighted inputs,
* adds a bias,
* then applies a nonlinear activation function.

## Multi-Layer Perceptron (MLP) or Fully Connected Network (FCN)

<div align="center">
  <img src="figs/neural_net2.jpeg" width="75%">
</div>


From Stanford's [cs231n](https://cs231n.github.io/convolutional-networks/).

* Multiple artificial neurons can act on the same inputs,. This defines
a _layer_. We can have more than one _layer_ until we produce one or more
outputs.

* The example above shows a network with _3 inputs_, two layers of neurons, each
with 4 neurons, followed by one layer that produces a single value output.

This architecture can be used as a binary classifier.

## FCN Detailed View

Here's a more detailed view of a fully connected network making biases explicit and with weight matrix and bias vector dimensions

<div align="center">
    <img src="./figs/udl_fcn_matrices.png" width="60%">
</div>


We can collect all the inputs into a column vector.

$$
\mathbf{x} =
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix}
$$

Then we can gather the weights for the first layer:

$$
h_1 = 
\textrm{a} \left[
\mathbf{\Omega}_0 \mathbf{x} + \boldsymbol{\beta}_0 \right] =
\textrm{a}\left[
\begin{bmatrix}
\omega_{11} & \omega_{12} & \omega_{13} \\
\omega_{21} & \omega_{22} & \omega_{23} \\
\omega_{31} & \omega_{32} & \omega_{33} \\
\omega_{41} & \omega_{42} & \omega_{43} \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3
\end{bmatrix} +
\begin{bmatrix}
\beta_1 \\
\beta_2 \\
\beta_3 \\
\beta_4
\end{bmatrix}
\right]
$$

where $a[z]$ is a nonlinear activation function, such as ReLU shown below.

<div align="center">
    <img src="figs/ReLU.png" width="75%">
</div>

The equation for which is.

$$
a[z] = \text{ReLU}[z] = 
\begin{cases}
0 & z < 0 \\
z & z \geq 0
\end{cases}
$$

or in pseudocode simply:

```python
x = max(0, x)
```



The nonlinear activation functions are crucial, because without them, the neural
network can be simplified down to a single linear system of equations, which is
severely limited in its representational power.

The subsequent layers can be defined similarly interms of the activations of the
previous activation functions.

$$
\begin{align}
\mathbf{h}_2 &= \textrm{a} \left[\mathbf{\Omega}_1 \mathbf{h}_1 + \boldsymbol{\beta} \right] \\
\mathbf{h}_3 &= \textrm{a} \left[\mathbf{\Omega}_2 \mathbf{h}_2 + \boldsymbol{\beta} \right] \\
\mathbf{h}_4 &= \mathbf{\Omega}_3 \mathbf{h}_3 + \boldsymbol{\beta} \\
\mathbf{y} &= \textrm{softmax} \left[ \mathbf{h}_4 \right]
\end{align}
$$

where the last function is a softmax, which limits the output to between 0 and 1,
and all the outputs sum up to 1.

$$
\mathrm{softmax}(y_i) = \frac{e^{y_i}}{\sum_{j=1}^K e^{y_j}}
$$

Here's a visualization.

<div align="center">
    <img src="figs/softmax.webp" width="60%">
</div>

From [TDS Post](https://medium.com/data-science/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1).


Much of neural network training and evaluation comes down to vector operations in which ndarray excels.


## Training a Neural Networks

* We won't go into all the details of training a network, but we'll show an example.

* The steps involve producing the outpur from some input -- **Forward Pass**

* Then updating the weights and biases of the network, taking the _partial derivatives_ of the entire model
  with respect to each parameter and then updating each parameter proportional to its partial derivative -- **Backward Pass**

### Simple weight update example

We'll build our understanding of the gradient by considering derivatives of single
variable functions.

Let's start with a quadratic function

$$
f(x) = 3x^2 - 4x +5.
$$

The derivative with respect to $x$ of our example is

$$
f'(x) = 6x-4.
$$

This is the asymptotic slope at any particular value of $x$.

<div align="center">
    <img src="figs/1d_derivative_plot.png" width="50%">
</div>

**Question:** Which way does $x$ need to move at $x = -2$ to _decrease_ $f(x)$? Positive or negative direction?

**Question:** What about at $x=2$? Positive or negative direction?
<br><br><br><br>

An easy way to update this is:

$$
x \leftarrow x - \alpha \frac{\partial f}{\partial x}
$$

where $\alpha$ is some small number (e.g. 0.01) which we call the **learning rate**.

### Minimizing the Network Loss

To train our network we want to minize some function that computes the difference
between our target values $t_i$ and what the network produces $y_i$ for each input $x_i$.

An appropriate loss function for classification models is the **categorical cross-entropy loss**.

The following image from the [Medium Post]() illustrates the full pipeline:

<div align="center">
    <img src="figs/medium_softmax_loss.webp" width="60%">
</div>

except in the figure $s$ is the model output and $y$ are the targets.

We won't go into details but a nice output is that

$$
\frac{\partial \ell_i}{\partial \mathbf{h}_4} = \mathbf{y} - \mathbf{t}
$$

## Forward propagation

* Weights would be represented by an `Array2<f32>` with dimensions: (# neurons $\times$ # inputs)
    * For example if you have 100 input neurons and 50 2nd layer neurons then you need a $50\times 100$ matrix of weights

* Bias vectors would be represented by an `Array2<f32>` with dimensions (# neurons $\times$ 1)

    

## Backwards propagation

* As mentioned, in practice the calculation of partial derivatives must be calculated starting with the end
  of the pipeline and then working backwards.

* The chain rule for differentiation in calculus gives a modular, scalable way of doing this calculation, again
  starting from the end and working backwards.

Just a reminder, we have

$$
\begin{align*}
\mathbf{f}_0 &= \mathbf{\Omega}_0 \mathbf{x} + \boldsymbol{\beta}_0  \\
\mathbf{h}_1 &= \textrm{a} \left[ \mathbf{f}_0 \right] \\
\mathbf{f}_1 &= \mathbf{\Omega}_1 \mathbf{h}_1 + \boldsymbol{\beta}  \\
\mathbf{h}_2 &= \textrm{a} \left[  \mathbf{f}_1  \right] \\
\mathbf{f}_2 &= \mathbf{\Omega}_2 \mathbf{h}_2 + \boldsymbol{\beta} \\
\mathbf{h}_3 &= \textrm{a} \left[ \mathbf{f}_2 \right] \\
\mathbf{h}_4 &= \mathbf{\Omega}_3 \mathbf{h}_3 + \boldsymbol{\beta} \\
\mathbf{y} &= \textrm{softmax} \left[ \mathbf{h}_4 \right]
\end{align*}
$$

We already did

$$
\begin{align*}
\frac{\partial \ell_i}{\partial \mathbf{h}_4} &= \mathbf{y} - \mathbf{t} \\
\frac{\partial \ell_i}{\partial \mathbf{\Omega}_3} &= \frac{\partial \ell_i}{\partial \mathbf{h}_4} \frac{\partial \mathbf{h}_4}{\partial \mathbf{\Omega}_3} = \frac{\partial \ell_i}{\partial \mathbf{h}_4} \mathbf{h}_3^T
\end{align*}
$$

And the process continues.

### Example Implementaiton

Let's look at an example implementation.

We'll start with some utility functions.

In [6]:
:dep ndarray = { version = "^0.15" }
:dep rand = { version = "0.8" }

use ndarray::prelude::*;
use rand::Rng;

// Helper function to populate arrays with random values
// We could use ndarray-rand crate to do this, but it doesn't seem to work in a Jupyter notebook
fn populate_array(arr: &mut Array2<f32>, m: usize, n: usize) {
    let mut rng = rand::thread_rng();
    for i in 0..m {
        for j in 0..n {
            arr[(i, j)] = rng.gen_range(-1.0..1.0);
        }
    }
}

// ReLU activation function
fn relu(x: &Array2<f32>) -> Array2<f32> {
    x.mapv(|x| if x > 0.0 { x } else { 0.0 })
}

// Derivative of ReLU
fn relu_derivative(x: &Array2<f32>) -> Array2<f32> {
    x.mapv(|x| if x > 0.0 { 1.0 } else { 0.0 })
}

// Softmax function
fn softmax(x: &Array2<f32>) -> Array2<f32> {
    let exp_x = x.mapv(|x| x.exp());
    let sum_exp_x = exp_x.sum();
    exp_x / sum_exp_x
}


Then we'll define a `NeuralNetwork` struct and associated methods.

In [ ]:

struct NeuralNetwork {
    input_size: usize,
    output_size: usize,
    weights1: Array2<f32>,
    biases1: Array2<f32>,
    weights2: Array2<f32>,
    biases2: Array2<f32>,
    learning_rate: f32,
}

impl NeuralNetwork {
    /// Create a shallow neural network with one hidden layer
    /// 
    /// # Arguments
    /// 
    /// * `input_size` - The number of input neurons
    /// * `hidden_size` - The number of neurons in the hidden layer
    /// * `output_size` - The number of output neurons
    /// * `learning_rate` - The learning rate for the neural network
    fn new(input_size: usize, hidden_size: usize, output_size: usize, learning_rate: f32) -> Self {
        let mut weights1 = Array2::zeros((hidden_size, input_size));
        let mut weights2 = Array2::zeros((output_size, hidden_size));

        // Initialize weights with random values
        populate_array(&mut weights1, hidden_size, input_size);
        populate_array(&mut weights2, output_size, hidden_size);

        let biases1 = Array2::zeros((hidden_size, 1));
        let biases2 = Array2::zeros((output_size, 1));

        NeuralNetwork {
            input_size,
            output_size,
            weights1,
            biases1,
            weights2,
            biases2,
            learning_rate,
        }
    }

    fn forward(&self, input: &Array2<f32>) -> (Array2<f32>, Array2<f32>, Array2<f32>) {
        // First layer
        let pre_activation1 = self.weights1.dot(input) + &self.biases1;
        let hidden = relu(&pre_activation1);

        // Output layer
        let pre_activation2 = self.weights2.dot(&hidden) + &self.biases2;
        let output = softmax(&pre_activation2);

        (hidden, pre_activation2, output)
    }

    fn backward(
        &mut self,
        input: &Array2<f32>,
        hidden: &Array2<f32>,
        pre_activation2: &Array2<f32>,
        output: &Array2<f32>,
        target: &Array2<f32>,
    ) {
        let batch_size = input.shape()[1] as f32;

        // Calculate gradients for output layer
        let output_error = output - target;
        
        // Gradients for weights2 and biases2
        let d_weights2 = output_error.dot(&hidden.t()) / batch_size;
        let d_biases2 = &output_error.sum_axis(Axis(1)).insert_axis(Axis(1)) / batch_size;

        // Backpropagate error to hidden layer
        let hidden_error = self.weights2.t().dot(&output_error);
        let hidden_gradient = &hidden_error * &relu_derivative(hidden);

        // Gradients for weights1 and biases1
        let d_weights1 = hidden_gradient.dot(&input.t()) / batch_size;
        let d_biases1 = &hidden_gradient.sum_axis(Axis(1)).insert_axis(Axis(1)) / batch_size;

        // Update weights and biases using gradient descent
        self.weights2 = &self.weights2 - &(d_weights2 * self.learning_rate);
        self.biases2 = &self.biases2 - &(d_biases2 * self.learning_rate);
        self.weights1 = &self.weights1 - &(d_weights1 * self.learning_rate);
        self.biases1 = &self.biases1 - &(d_biases1 * self.learning_rate);
    }

    fn train(&mut self, input: &Array2<f32>, target: &Array2<f32>) -> f32 {
        // Forward pass
        let (hidden, pre_activation2, output) = self.forward(input);

        // Calculate loss (cross-entropy)
        let epsilon = 1e-15;
        let loss = -target * &output.mapv(|x| (x + epsilon).ln());
        let loss = loss.sum() / (input.shape()[1] as f32);

        // Backward pass
        self.backward(input, &hidden, &pre_activation2, &output, target);

        loss
    }
}


In [ ]:

fn main() {
    // Create a neural network
    let mut nn = NeuralNetwork::new(6, 6, 4, 0.01);

    // Create sample input
    let mut input = Array2::zeros((nn.input_size, 1));
    populate_array(&mut input, nn.input_size, 1);

    // Create sample target
    let mut target = Array2::zeros((nn.output_size, 1));
    populate_array(&mut target, nn.output_size, 1);
    
    // Calculate initial cross entropy loss before training
    let (_, _, initial_output) = nn.forward(&input);
    let epsilon = 1e-15;
    let initial_loss = -&target * &initial_output.mapv(|x| (x + epsilon).ln());
    let initial_loss = initial_loss.sum() / (input.shape()[1] as f32);
    println!("Initial loss before training: {}", initial_loss);

    
    // Train for one iteration
    let loss = nn.train(&input, &target);
    //println!("Loss: {}", loss);

    // Calculate loss after training
    let (_, _, final_output) = nn.forward(&input);
    let epsilon = 1e-15;
    let final_loss = -&target * &final_output.mapv(|x| (x + epsilon).ln());
    let final_loss = final_loss.sum() / (input.shape()[1] as f32);
    println!("loss after training: {}", final_loss);

}

We can now run an iteration of a forward pass and backward pass.

You can repeatedly run this cell to see the loss decreasing.

In [8]:
main()

Initial loss before training: 1.0153453
loss after training: 0.8079079


()

## Has anyone implemented neural networks in Rust?

See for example [candle](https://github.com/huggingface/candle). Perhaps a pun on "Torch" from "PyTorch".

But the point isn't to use an existing package but learn how to build a simple one from basic linear algebra principles.

### All of this and more...

* Quick start: https://github.com/rust-ndarray/ndarray/blob/master/README-quick-start.md
* Numpy equivalence: https://docs.rs/ndarray/latest/ndarray/doc/ndarray_for_numpy_users/index.html
* Linear Algebra: https://github.com/rust-ndarray/ndarray-linalg/blob/master/README.md

# In class poll

https://piazza.com/class/m5qyw6267j12cj/post/441
